

<H1>Методы интеллектуального анализа текстов</H1>
<H2>Тема 6. Извлечение именованных сущностей</H2>



### Задание 3:
Улучшить качество базовой предсказательной CRF-модели на тестовой выборке за счет добавления и модификации признаков.

In [1]:
! wget https://www.dropbox.com/s/iuwsx5pmfhkk0w2/ner_dataset.csv

--2022-10-16 14:47:36--  https://www.dropbox.com/s/iuwsx5pmfhkk0w2/ner_dataset.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/iuwsx5pmfhkk0w2/ner_dataset.csv [following]
--2022-10-16 14:47:37--  https://www.dropbox.com/s/raw/iuwsx5pmfhkk0w2/ner_dataset.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbcfc76ca5c2fd264153d994277.dl.dropboxusercontent.com/cd/0/inline/Bu6JWj8RX8_4zzgOWakpmy80QtyCbGH1u2qDuZQkHT_Thj1xTM6axI7JdYkmVXi3yW9WqQKUrZvvDEOFUPJijOtUNKcaFjia-FR34NQIVg2FWA3dezfLRPQ1_OsyvfGbjQeDR3R-8Oh4rcu1XTjy_REzB2okL9aCg1Igl4dfc79dgg/file# [following]
--2022-10-16 14:47:37--  https://ucbcfc76ca5c2fd264153d994277.dl.dropboxusercontent.com/cd/0/inline/Bu6JWj8RX8_4zzgOWakpmy80QtyCbGH1u2qDuZQkHT_Thj1xTM6axI7JdYkmVXi3yW9WqQK

Устанавливаем старую версию ***scikit-learn***, ибо на более свежих возникает ошибка атрибутов при работе с ***sklearn-crfsuite***

In [2]:
! pip install -U 'scikit-learn<0.24'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 5.2 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


In [4]:
! pip install sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 4.7 MB/s 


Импортируем нужные библиотеки.

In [18]:
import pandas as pd
import nltk
import sklearn_crfsuite
from sklearn_crfsuite import metrics
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

In [6]:
data = pd.read_csv("ner_dataset.csv", encoding="latin1")
data = data.fillna(method="ffill")

agg_func = lambda s: [[w, p, t] for w, p, t in zip(s["Word"].values.tolist(),
                                                   s["POS"].values.tolist(),
                                                   s["Tag"].values.tolist())]
grouped = data.groupby("Sentence #").apply(agg_func).values.tolist()

X_list = [[word[:2] for word in sentence] for sentence in grouped]
y_list = [[word[2] for word in sentence] for sentence in grouped]

In [7]:
data_train, data_test, y_train, y_test = train_test_split(X_list, y_list, test_size=0.2, random_state=1337)

In [10]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [30]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        # 'stopword': True if word in stop_words else False,
        # 'bias': 1.0,
        # 'word[-3:]': word[-3:],
        # 'word[-2:]': word[-2:],
        # 'postag[:2]': postag[:2]
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            # '-1:postag': postag1,
            # '-1:postag[:2]': postag1[:2]
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            # '+1:postag': postag1,
            # '+1:postag[:2]': postag1[:2]
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [31]:
X_train = [sent2features(s) for s in data_train]
X_test = [sent2features(s) for s in data_test]

Инициализация и обучение модели

In [32]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [33]:
labels = list(crf.classes_)
labels.remove('O') # Удаляем сегменты, которые не принадлежат к какой-либо именованной сущности

In [34]:
labels

['B-org',
 'I-org',
 'B-gpe',
 'B-tim',
 'I-tim',
 'B-per',
 'I-per',
 'B-geo',
 'I-geo',
 'B-eve',
 'B-nat',
 'I-eve',
 'I-nat',
 'B-art',
 'I-art',
 'I-gpe']

Результаты если раскомментировать дополнительные признаки в функции **word2features**

In [35]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

0.8592386330055357

Результаты если функцию **word2features** оставить без изменений

In [29]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

0.8584531001802781

Как мы можем видеть, если мы используем дополнительные признаки, то точность модели немного улучшается. В целом, точность модели относительно неплоха.